In [ ]:
pip install sklearn

In [ ]:
import pandas as pd
import numpy as np
import random

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement sklearn
ERROR: No matching distribution found for sklearn


In [ ]:
movies_dataset=pd.read_csv('movies.csv')
ratings_dataset=pd.read_csv('ratings.csv')

In [ ]:
print(movies_dataset.shape )
print(ratings_dataset.shape )

(9125, 3)
(100004, 4)


In [ ]:
movies_dataset.head(5)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [ ]:
ratings_dataset.head(5)

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [ ]:
movies_dataset['new'] = movies_dataset.genres.str.split('|')
movies_dataset['new']

0       [Adventure, Animation, Children, Comedy, Fantasy]
1                          [Adventure, Children, Fantasy]
2                                       [Comedy, Romance]
3                                [Comedy, Drama, Romance]
4                                                [Comedy]
                              ...                        
9120                          [Adventure, Drama, Romance]
9121                 [Action, Adventure, Fantasy, Sci-Fi]
9122                                        [Documentary]
9123                                             [Comedy]
9124                                        [Documentary]
Name: new, Length: 9125, dtype: object

In [ ]:
movies_dataset['year'] = movies_dataset.title.str.extract('(\(\d\d\d\d\))',expand=False)
movies_dataset['year'] = movies_dataset.year.str.extract('(\d\d\d\d)',expand=False)

In [ ]:
movies_dataset['movie_name']= movies_dataset.title.str.replace(' (\(\d\d\d\d\))', '', regex=True)

In [ ]:
movies_dataset['new']=movies_dataset['new'].apply(' '.join)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
tfid=TfidfVectorizer(stop_words='english')
matrix=tfid.fit_transform(movies_dataset['new'])

In [ ]:
#Compute the cosine similarity of every genre
from sklearn.metrics.pairwise import cosine_similarity


In [ ]:
cosine_sim=cosine_similarity(matrix,matrix)

In [ ]:
#Movie name and movie id
movies_dataset = movies_dataset.reset_index()
titles = movies_dataset['movie_name']
indices = pd.Series(movies_dataset.index, index=movies_dataset['movie_name'])

In [ ]:
movies_dataset.head(2)

,index,movieId,title,genres,new,year,movie_name
0,0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Adventure Animation Children Comedy Fantasy,1995,Toy Story
1,1,2,Jumanji (1995),Adventure|Children|Fantasy,Adventure Children Fantasy,1995,Jumanji


In [ ]:
def recommendataion(movie):
    result=[]
    ind=indices[movie]
    sim_scores=list(enumerate(cosine_sim[ind]))
    sim_scores=sorted(sim_scores,key=lambda x:x[1],reverse=True)
    movie_id=[i[0] for i in sim_scores]
    print('The Movie You Should Watched Next Are --')
    print('ID ,   Name ,  Average Ratings , Year ')
    count=0
    for id in range(0,len(movie_id)):
        if(ind != movie_id[id]):
            ratings=ratings_dataset[ratings_dataset['movieId']==movie_id[id]]['rating']
            avg_ratings=round(np.mean(ratings),2)
            if(avg_ratings >3.5):
                count+=1
                print(f'{movie_id[id]} , {titles[movie_id[id]]} ,{avg_ratings}')
                result.append([titles[movie_id[id]],str(avg_ratings)])
            if(count >=10):
                    break

    print('Wait!! i am telling your recommendation')
    return result

In [ ]:
query_index = np.random.choice(movies_dataset.movie_name)
print(query_index)

Great Santini, The


In [ ]:
result=recommendataion(query_index)

The Movie You Should Watched Next Are --
ID ,   Name ,  Average Ratings , Year 
13 , Nixon ,3.94
25 , Othello ,3.74
30 , Dangerous Minds ,4.05
41 , Restoration ,4.02
52 , Georgia ,3.64
53 , Home for the Holidays ,5.0
58 , Mr. Holland's Opus ,4.0
116 , Angela ,4.75
146 , Feast of July ,4.33
154 , Kids ,3.81
Wait!! i am telling your recommendation
